# Mini Competition Richter's Predictor

Import modules and data

In [ ]:
# Modules
import pandas as pd

In [4]:
# Main Scripts
from script import imports

Data Analysis

Data Cleaning

Visualization

Tree

Evaluation

Export